In [24]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

using namespace std;

struct Contact {
  char id[50];
  char name[50];
  char number[50];
};

__device__ bool check(char* str1, char* str2, int len) {
  for(int i = 0; str1[i] != '\0'; i++) {
    int j = 0;
    while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
      j++;
    }
    if(j == len-1) {
      return true;
    }
  }
  return false;
}

__global__ void searchPhonebook(Contact* phonebook, int num_contacts, char* search_name, int name_length) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if(idx < num_contacts) {
    if(check(phonebook[idx].name, search_name, name_length)) {
      printf("%s %s %s\n", phonebook[idx].id, phonebook[idx].name, phonebook[idx].number);
    }
  }
}


int main(int argc, char* argv[]) {

  if(argc != 3) {
    cerr << "Usage: " << argv[0] << "  " << endl;
    return 1;
  }

  string search_name = argv[1];
  int num_threads = atoi(argv[2]);
  // Mount Google Drive and copy the location
  string file_name = "latest_dataset.txt";

  vector <Contact> phonebook;

  ifstream file(file_name);
  if(!file.is_open()) {
    cerr << "Error opening file: " << file_name << endl;
    return 1;
  }
  else {
    Contact contact;
    string line;
    while(getline(file, line)) {
      // Format: "id","name"."phone_number"
      int pos = line.find(",");
      strcpy(contact.id, line.substr(1, pos-2).c_str());
      line = line.substr(pos+1);
      pos = line.find(",");
      strcpy(contact.name, line.substr(1, pos-2).c_str());
      strcpy(contact.number, line.substr(pos+2, line.size()-pos-4).c_str());
      phonebook.push_back(contact);


      /*

      // Format: "name"."phone_number"
      int pos = line.find(",");
      // Extract name (without the quotes)
      strcpy(contact.name, line.substr(1, pos - 2).c_str());

      // Extract number (also without quotes)
      strcpy(contact.number, line.substr(pos + 2, line.size() - pos - 4).c_str());

      phonebook.push_back(contact);
      */
    }
    file.close();
  }
  int num_contacts = phonebook.size();
  Contact* device_phonebook;
  cudaMalloc((void**)&device_phonebook, sizeof(Contact)*num_contacts);
  cudaMemcpy(device_phonebook, phonebook.data(), sizeof(Contact)*num_contacts, cudaMemcpyHostToDevice);

  int name_length = search_name.length() + 1;
  char* device_search_name;
  cudaMalloc((void**)&device_search_name, name_length);
  cudaMemcpy(device_search_name, search_name.c_str(), name_length, cudaMemcpyHostToDevice);

  for(int i = 0; i < num_contacts; i += num_threads) {
    int thread_count = min(num_contacts-i, num_threads);
    searchPhonebook<<<1, thread_count>>>(device_phonebook + i, thread_count, device_search_name, name_length);
    cudaDeviceSynchronize();
  }

  cudaFree(device_phonebook);
  cudaFree(device_search_name);

  return 0;
}

Overwriting phonebook_search.cu


In [25]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [46]:
!time ./phonebook_search SAKI 2

8 SAKIA RAHMAN 017 22 30
190 MD.SAKIL 011 17 56
695 MD.SAKIB 015 08 24
955 MD. SAKIN 017 61 68
1226 SAKIBUL HASAN 013 72 01
1884 SAKINA BEGUM 017 26 78
1934 SAKIB HOSSAIN 016 64 16
2055 MD. SAKIL AHAMMED 011 18 80
2152 MD.SAKIB HOSSEN 012 52 50
2322 MD. SAKIL 015 38 52
2696 MD. SAKIL 011 52 45
2700 MD. SAKIL 016 01 36
2962 MD. SAKIL 016 00 28
3154 MD. SAKIB 015 17 04
3441 MD. SAKIB 018 34 15
3442 MD. SAKIB 011 78 06
3907 MD. SAKIL 012 05 61
4049 MD. SAKIL 013 86 77
4175 MD. SAKIL 012 83 27
4476 MD. SAKIL 012 27 48

real	0m0.158s
user	0m0.050s
sys	0m0.102s
